## __Pick up Gaia EDR3 Data__ ##

__You can obtain the required files from the following website：__
https://warwick.ac.uk/fac/sci/physics/research/astro/research/catalogues/gaiaedr3_wd_main.fits.gz  
 (from article,  $\textit{A catalogue of white dwarfs in Gaia EDR3}$  

* __the following codes are to create a substitutable, simplified *.csv__  
( only included ra, dec, Pwd, mag and WDJ_name  
( the simplified file is in a 'temp' folder in the same folder as your file, gaiaedr3_wd_main.fits.gz

In [1]:
# the needed module import
# used to read the fits
from astropy.io import fits 

# used to desgin functions to simplify the code
from tqdm import tqdm, trange # not necessary, just a visible progress bar

# to create a new csv
import csv
import os
from pathlib import Path

In [2]:
path = 'C:/Users/Administrator/Desktop/大创/Data' #the path you saved the gaiaedr3_wd_main.fits.gz
name_list = ['WDJ_name','ra','dec','Pwd','phot_g_mean_mag_corrected']
Pwd_limitation = 0.75 #the minimum of Pwd
mag_limitation = [18, None]#the range of mag

In [3]:
class InputError(Exception):
    pass
def Gaia_EDR3_Data_Split(path,
                        name_list,
                        Pwd_limitation,
                        mag_limitation,
                        next_folder = 'Gaia',
                        file_create = True,
                        return_data = False):
    
    #through Pwd and phot_g_mean_mag_corrected
    if 0 > Pwd_limitation or Pwd_limitation >= 1:
        raise InputError ('Pwd_limitation should in [0, 1)')
    if len(mag_limitation) != 2:
        raise InputError ('mag_limitation should be a list with 2 element or NoneType')
    
    if 'WDJ_name' not in name_list:
        name_list.insert(0 , 'WDJ_name')
    elif name_list[0] != 'WDJ_name':
        temp = name_list[0]
        name_list[0] = 'WDJ_name'
        for i in range(1, len(name_list)):
            if name_list[i] == 'WDJ_name':
                name_list[i] = temp
                temp = None
                break
    if path[len(path) - 1] == '/':
        path = path + 'gaiaedr3_wd_main.fits.gz'
    elif Path(path).parts[len(Path(path).parts)-1] != 'gaiaedr3_wd_main.fits.gz':
        path = path + '/' + 'gaiaedr3_wd_main.fits.gz'
        
    try:
        fits_file = fits.open(path)
    except FileNotFoundError:
        print('Wrong path of gaiaedr3_wd_main.fits.gz')
    else:
        fits_file_num = fits_file[1].header['NAXIS2']   # the total number of WDs in GaiaEDR3_WD_main.fits
        fits_file_data = fits_file[1].data   # pre-load the data
        fits_file.close()
        
        Pwd = fits_file_data.field('Pwd')
        Mag = fits_file_data.field('phot_g_mean_mag_corrected')
        
        data = []
        for j in trange(0 , fits_file_num):
            if Pwd[j] > Pwd_limitation:
                if mag_limitation[0] is not None and mag_limitation[1] is not None:
                    if mag_limitation[0] < Mag[j] <= mag_limitation[1]:
                        temp = []
                        for i in range(len(name_list)):
                            temp.append(fits_file_data.field(name_list[i])[j])
                        data.append(temp)
                
                elif mag_limitation[0] is not None or mag_limitation[1] is not None:
                    if mag_limitation[0] is not None:
                        if mag_limitation[0] < Mag[j]:
                            temp = []
                            for i in range(len(name_list)):
                                temp.append(fits_file_data.field(name_list[i])[j])
                            data.append(temp)
                    
                    else:
                        if Mag[j] <= mag_limitation[1]:
                            temp = []
                            for i in range(len(name_list)):
                                temp.append(fits_file_data.field(name_list[i])[j])
                            data.append(temp)
                    
                else:
                    temp = []
                    for i in range(len(name_list)):
                        temp.append(fits_file_data.field(name_list[i])[j])
                    data.append(temp)
        if file_create:
            Gaia_Simplified_File_Create(path, next_folder, mag_limitation, name_list, data)
        if return_data:
            return data

In [4]:
def Gaia_Simplified_File_Create(path,
                                next_folder,
                                mag_limitation,
                                name_list,
                                data):
    path = path.replace('gaiaedr3_wd_main.fits.gz' , next_folder)
    if not os.path.exists(path):
        os.makedirs(path)
        
# 以写方式打开文件。注意添加 newline=""，否则会在两行数据之间都插入一行空白。
    with open(path+'/selected_data_{0}_{1}.csv'.format(mag_limitation[0] , mag_limitation[1]), mode="w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(name_list)
        writer.writerows(data)

In [5]:
Gaia_EDR3_Data_Split(path, name_list, Pwd_limitation, mag_limitation)

100%|██████████| 1280266/1280266 [00:51<00:00, 24716.91it/s]
